# SNARE-Protein Project

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS
import urllib
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

## Introduction

## If Any Known Proteins From SNARE Family Belongs to Mycobacterium Tuberculosis

### 1. Get SNARE family protein IDs from InterPro

In [14]:
# downloadProteinID function 
# input: 
#   None
# output:
#   the name of the file contains a list of proteins IDs downloaded from InterPro
def downloadProteinID ():
    url = "http://www.ebi.ac.uk/interpro/entry/IPR010989/proteins-matched?taxonomy=2&export=ids"
    r = requests.get(url, allow_redirects=True)
    open("SNARE_proteins_bacteria.txt", 'wb').write(r.content)
    return "SNARE_proteins_bacteria.txt"

# The file of protein ids download from InterPro
proteinIDs = downloadProteinID ()

### 2. Get organism names from Uniport

In [15]:
# searchUniport_stable function
# input:
#   filename: string
#             the filename of a list of Protein ids 
# output:
#   a file with all the organism names matched to the protein ids and the
#   corresponding freqency of apperance
# SideNote: This function accomplish the smae goal as searchUniport_fast, but
#           this function accomplish the goal with uniprot api, where the 
#           result is stable but rather slow, which could take up to 10 mins.
def searchUniport_stable (filename):
    outFile = open("organism.txt", "w")
    proteinIDs = open(filename, "r")
    orgDic = {}
    for protein in proteinIDs:
        url = "https://www.uniprot.org/uniprot/?query=id:" + protein + "&columns=organism&format=tab"
        result = requests.get(url).content
        organism = str(result).split("\\n")[1]
        if organism in orgDic:
            orgDic[organism] = orgDic[organism] + 1
        else:
            orgDic[organism] = 1
    for org_freqs in sorted(orgDic, key=orgDic.get, reverse=True):
        outFile.write(str(orgDic[org_freqs]) + " " + org_freqs + "\n")

In [16]:
# searchUniport_fast function
# input:
#   filename: string
#             the filename of a list of Protein ids 
# output:
#   a file with all the organism names matched to the protein ids and the
#   corresponding freqency of apperance
# SideNote: This function accomplish the smae goal as searchUniport_stable, but
#           accomplish the goal with python's BeautifulSoup package, where the 
#           result could be unstable but faster than using Uniport API. 
def searchUniport_fast (filename):
    outFile = open("organism.txt", "w")
    proteinIDs = open(filename, "r")
    orgDic = {}
    for protein in proteinIDs:
        text = requests.get('http://www.uniprot.org/uniprot/' + protein).text
        soup = BS(text)
        title = soup.head.title.text
        organism = title.split(" - ")[2]
        if organism in orgDic:
            orgDic[organism] = orgDic[organism] + 1
        else:
            orgDic[organism] = 1
    for org_freqs in sorted(orgDic, key=orgDic.get, reverse=True):
        outFile.write(str(orgDic[org_freqs]) + " " + org_freqs + "\n")

Get organism names from Uniport. This could take a while depends on the function choice. 

In [17]:
searchUniport_fast(proteinIDs)

### 3. Search through the organism names to see if it contains Mycobacterium Tuberculosis

In [18]:
# queryOrg function
# input: 
#   queryName: string
#              the name of organism we want check if it's in the organism list
#   orgFile: string
#            the name of the file output by searchUniport function, which contains
#            all the organism names that have a specific protein fanmily
# output:
#   a boolean that tells if the query organism is in the organisms list
def queryOrg (queryName, orgFile):
    f = open(orgFile, "r")
    for org in f:
        lowerOrg = org.lower()
        qLower = queryName.lower()
        if qLower in lowerOrg:
            return True
    return False

In [20]:
# Determine if TB is in the orgnanism list
query = "Mycobacterium tuberculosis"
organism = "organism.txt"
result = queryOrg(query, organism)
print ("Mycobacterium tuberculosis is in the name list: " + str(result))

Mycobacterium tuberculosis is in the name list: False


### 4. Result and Conclusion

## BLAST SNARE-like proteins against Mycobacterium tuberculosis

### 1. Import Data

Import a list of all (most) predicted "Incs" from C. trachomatis from the paper ***Expression and Localization of Predicted Inclusion Membrane Proteins in Chlamydia trachomatis, Mary M. Weber ect.*** Store the tags and 3 corresponding strains in `C.trachomatis_predict_Incs.txt`

Change the locus tag for D/UW-3/CX strain in order to match NCBI locus tag format and store the changed tags into DUW-3CX_predict_Incs.txt file using unix command:
```shell
cut -d " " -f 1 C.trachomatis_predict_Incs.txt | sed s/CT/CT_/g > DUW-3CX_predict_Incs.txt
```
Get the locus tag for L2/434/Bu strain and A/HAR-13 strain using similar commands:
```shell
cut -d " " -f 2 C.trachomatis_predict_Incs.txt > L2434Bu_predict_Incs.txt
cut -d " " -f 3 C.trachomatis_predict_Incs.txt > AHAR13_predict_Incs.txt
```

### 2. Set up Entrez Direct to perfrom BLAST search

Follow the direction from NCBI website [here](https://www.ncbi.nlm.nih.gov/books/NBK179288/), install `Entrez Direct: E-utilities` with UNIX Command Lines.

First, run the following code on terminal:
```shell
cd ~
  /bin/bash
  perl -MNet::FTP -e \
    '$ftp = new Net::FTP("ftp.ncbi.nlm.nih.gov", Passive => 1);
     $ftp->login; $ftp->binary;
     $ftp->get("/entrez/entrezdirect/edirect.tar.gz");'
  gunzip -c edirect.tar.gz | tar xf -
  rm edirect.tar.gz
  builtin exit
  export PATH=${PATH}:$HOME/edirect >& /dev/null || setenv PATH "${PATH}:$HOME/edirect"
  ./edirect/setup.sh
```
Then, add the command edirect to soucre.sh so that it can be run in any path:

```shell
echo "source ~/.bash_profile" >> $HOME/.bashrc
echo "export PATH=\${PATH}:/Users/xuqiantan/edirect" >> $HOME/.bash_profile
```


### 3. Use esearch to get the sequence of the tagged locus

Use the unix command below to get tagged locus protein sequences for each strain and store all the sequence in `Chlamydia_protein_seq.fasta` for BLAST search:

```shell
for file in DUW-3CX_predict_Incs.txt L2434Bu_predict_Incs.txt AHAR13_predict_Incs.txt
do
    input=${file}
    while IFS= read -r line
    do
        echo ${line}
        esearch -db protein -query ${line} < /dev/null | efetch -format fasta >> Chlamydia_protein_seq.fasta 
    done < "$input"
done
```  

Common code errors for using `esearch` in Unix command for loops can be found [here](https://www.biostars.org/p/217450/)

### 4. Perform BLAST search on the protein sequence

Use import_queries to import query sequences from `Chlamydia_protein_seq.fasta`

In [2]:
def import_queries(fasta_file):
    queries = []
    f = open(fasta_file, "r")
    q = ""
    for line in f:
        if line[0] == ">":
            queries.append(q)
            q = ""
        else:
            q = q+line.strip()
    queries.remove("")
    # remove duplicates from queires
    queries = list(dict.fromkeys(queries))
    return queries

queries = import_queries("Chlamydia_protein_seq.fasta")   
print(len(queries))

154


Use `Biopython package (version 1.73)` to perform BLAST search for the queries. Install Biopython using the unix command below:
```shell
pip install biopython
```
The general usage of Biopython package can be found [here](http://biopython.org/DIST/docs/tutorial/Tutorial.html). Performing BLAST search is in Chapter 7. Import the package before using. 

In [3]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

Use `NCBIWWW` from Biopython to perform search and `NCBIXML` from Biopython to parse the BLAST result into readble text. Filter the BLAST search result by checking if "Mycobacterium tuberculosis" or "Chlamydia caviae" is contained in the `alignment.title` and write all eligible results into `blast_control_search.txt`. "Chlamydia caviae" is used as a positive control for the BLAST search to ensure the BLAST search is performing correctlly. 

Keep in mind: based on the Biopython tutorial, we could use `result_handle.read()` to read the BLAST output only once and calling `result_handle.read()` again returns an empty string. For reference, the following code could be used to store result_handle so that `result_handle` could be read multiple times in the future.
```shell
    output = open("blast_result.xml", "w")
    output.write(result_handle.read())
    result_handle.close()
    result_handle = open("blast_result.xml")
```

**Note**: running the cell below is very time consuming and could take up to **9 hours** to finish running. 

In [4]:
count = 1
result = open("blast_control_search.txt", "w")

# run blast for each queries
for q in queries:
    print("searching for " + q)
    
    # choose refseq as database
    result_handle = NCBIWWW.qblast("blastp", "refseq_protein", q)
    
    # parse the BLAST result into readble text
    blast_record = NCBIXML.read(result_handle)
    result.write("result for " + q + "\n")
    for alignment in blast_record.alignments:
        # print out a small portion of the nonlevent result to make sure process is going
        if ("Mycobacterium tuberculosis" in alignment.title) or ("Chlamydia caviae" in alignment.title):
            for hsp in alignment.hsps:
                result.write("****Alignment****\n")
                result.write("sequence:" + alignment.title + "\n")
                result.write("length:" + str(alignment.length) + "\n")
                result.write("e value:" + str(hsp.expect) + "\n")
                result.write(hsp.query[0:75] + "..."+ "\n")
                result.write(hsp.match[0:75] + "..."+ "\n")
                result.write(hsp.sbjct[0:75] + "..."+ "\n")
    
    # print information after each search ends
    print("search " + str(count) + " ends \n")
    count += 1

result.close()

searching for MTPVTPVPPQSPQQVKGLLSRFLTAPDRHPKLRYVYDIALIAISILCIVSIILWTQGSGLALFAIAPALAIGALGVTLLVSDLAESQKSKEIADTVAAVSLPFILTGTAAGLMFSAIAVGGGAVILANPLFLMGSMTLGFALMSLHRVTYQYLSNREQWKQQKKLEQVELAAWESHLPKESKSSALEEVRYSPRLMKRGKTWRKRAIRRKNYTPIPLVDKTLQTMQPDALFSSTTTHSTDSEQILTSVSPQSSDTESSSSSSFHTPPNSDKELSDSNSSDSSSSSEYMDALETVAAGDVSGITPPSKPSSSPKTTRRVVKLSRSERNAQHHRNKDQEQRQDSSESSEEDSSSDSSQKKKPSRK
search 1 ends 

searching for MPSTVAPIKGQDHFLNLVFPERVAAAYMSPLAQKYPKAALSIASLAGFLLGILKLITFPVLCAAGLFVFPIRGLISCLFHKSFQGCSGYVLATFLSLFSLALTIVGIVSCITWAPGFIFPMISVSIAFATVETCFQIYTHLFPALEHKPSSSLKIEIAAAKLPRSSSAPDLNYPSLPTQSASPSQRFSA
search 2 ends 

searching for MFPIECHTLQTSFKQVLSLVAEKITTKAFVIFSVLFSLIIGFIASCGFLFAGPPAFIASGLCFALLVSVVSFFGCQKLIPYGIQHLMSYVKSIPSLSSSLIDFLKTESKSISSLYPNPGLKECFKGASPKYKKFFFDHPEKLLSAAFTDWTPQIIPSDSGQPRTIILSHSSLPFSLTLSTLDFETLHTHLIKSNALTCRVGYAHQLPSGNPVMREAKEGVLQQHYDTGNETFFISIQESKQLQQEELFKKLFSHYAQITEHNLSNEILLLEPLKTPLHTPKARTLELLALFCALEQLRYTKVADWRTKKLAPIFPLDYEDFFTLFMKKQHYTLPGNVSNMRILSPVRPVSETALTTIIISGLEEEDKLGLLG

search 27 ends 

searching for MVSLALGTSNGVEANNGINDLSPAPEAKKTGSGLCYKISAVAALVLGLLAAAGGAVVLALFCTFAPPLFFYAGVALVALGAVILGVGVSNTCSCCLRSRKIEAHKQLILQQKEEISQLEQQLAKALKELNTKYPASLLERRDLRENLKAWQSCCLNLEKEVRDLLTKLGGYQERLKVLPAKEKQIEELKAMLEHYSRICHERGELINLLKTANKKLSKESEKLLFNYKAHRDVCLGEKVLVKSVNLIDLDPKSDSSDGDDDDGFNYGSRV
search 28 ends 

searching for MSFVGDSVPLRSYMPEAPLVDSASKARVSCCSERIAVLALGILSILFIVTGAALFIGAGWTTLPMIDVVVTLVVFGSVMLGAVLTRISSYGGEPKKVSLDRFVLENERQGFLDKQRLADISKEEIALAKQQIEEEKEAILHSIFPND
search 29 ends 

searching for MANNSFIHRSKTYQLFVVVLTSLLAALGGVFLCLGGVYSSLVLGVVGGAAIIGSCIGAFGLVSYLLSVIRNSDQLLQEAKESARKISSHYRVLETQKNREIGLLEERVNMLDGFYAKFHGWD
search 30 ends 

searching for MLAFFLRNRGAMFNISFCCNSSKPLRADHTETIGAQTTTSRKEQLLAIGALVLGVLAVLGGALLLLFSGSVVSLFAPILSLLAMTLGSACIGGSLVYMYGFSLKPTRLPSESSELAPEAVTPGLVLSYQELLYEAEEDLKEVEGLLAQKSKDLELAQKKIEQLQSGLKCVLEESLI
search 31 ends 

searching for MSYLFCSSCAPTLESPAELCLYKTHIYCKRRGNIEFAVSLGIFAILSCVALLCLLCGGSSLVFAGLGIGAIMIGSVALGVGLTFLYWSCSRGLQNRIRTNILASSDSSSLFSSKSDFSLEFELNEA

search 58 ends 

searching for MNSGMFPFTFFLLYICLGMLTAYLANKKNRNLIGWFLAGMFFGIFAIIFLLILPPLPSSTQDNRSMDQQDSEEFLLQNTLEDSEIISIPDTMNQIAIDTEKWFYLNKDYTNVGPISIVQLTAFLKECKHSPEKGIDPQELWVWKKGMPNWEKVKNIPELSGTVKDE
search 59 ends 

searching for MNSNIEYRQYRIDILSCFICLLMMVWTLVSIKLGDSLGGIIPGCLGYLLAKRKHRRPVRWFFLTFFFGIASGIFLVVLHPKQK
search 60 ends 

searching for MTTLPNNCTSNSNSINTFTKDIEMAKQIQGSRKDPLAKTSWIAGLICVVAGVLGLLAIGIGGCSMASGLGLIGAIVAAVIVAVGLCCLVSALCLQVEKSQWWQKEFESWIEQKSQFRIVMADMLKANRKLQSEVEFLSKGWSDDTAVHKEDVTKYEQVVEEYAEKIMELYEETGVLTIEKINLQKEKKAWLEEKAEMEQKLTTVTDLEAAKQQLEEKVTDLESEKQELREELDKAIENLDEMAYEAMEFEKEKHGIKPGRRGSI
search 61 ends 

searching for MGLYDRDYAQDSRLPGTFSSRVYGWMTAGLAVTALTSLGLYATGAYRALFPMWWIWCFATLGVSFYIQAQIQKLSVPAVMGLFLAYSILEGMFFGTLVPVYAAQFGGGVVWAAFGSAGIIFGLSAAYGAFTKNDLTQIHRILMLALVGLVVISLAFLIVSLFTPMPLLYLLICYLGLIIFVGLTVVDAQSIRRVARSVGDHGDLSYKLSLIMALQMYCNVIMIFWYLLQIFASSDKRR
search 62 ends 

searching for MRVIFPDKYKQTPFLGKALKQLPLLVLVTSCSAPFIAFFLQYFFQVRGPIEWLALSIKGIHQHYFWQWLTYPLVTADTLKLGDLRSLEIT

search 85 ends 

searching for MQSVGQEASQNTLSWTIRPRIPSIIQGSKEVSLALFVLGTVLAIVGACAAAVGGAFSVCLGALFLGGVVLATGLLLAVLEFCHIRSSREKYVILTKQDLFKEPVIQEEQATPLIEEASYTCEPGIPLSGPEEVQQERPVILQKDLDLSHVPKYIAVGSHVVELVKAGKIGRNGELLLEEGIDTDQNFVRVWDELFILGQRGEVVRLDGFCCKVLPKTSKSESINDLVSNDC
search 86 ends 

searching for MVSMSLNLPPAEVRLRPVTAKSFPNSSAWRATQKRITGCSLNASSYRPSSARVISFVVGVLVVLGWICCRIAYLANSRVLTFPKMFAIAILPFILFGGGLAILFRIAGKVDVLYGKKIQPFISRQWERVILCEKEGEFIRPIQEPDMYMDVSLLDRTGSGIAPVYTYPPMDARTVICVVTSILLPIFSIVRMLYNIFRFFIVPFYILFQMVRQNYQTDIPKEERFVCSDIVREMTRSLLQAVKAPFYGAACYLANLYGLLNPLSGRVVLASLERDWNNDVIRSRGVWGIFCEKNYMFEGGGTRSGLGQNAWYLLGCFQPAQLFLLKDGVIISGARPSIQSFPESKEYLASFLYGAVPGRLASF
search 87 ends 

searching for MATTVNPNYSLPFCEKMVSSATLAAHSFFNHLKLLIPLLVGYFCILLGALILTGVITTIPSIAASYFLSLGVTLVIAGAGLCAAFKRPLFSVTQSKASTLLHISCD
search 88 ends 

searching for MRTDSPLNPPDSTRGVFQFLETQCDRAMARSRQSQFIGLVSAVAAAALLLLLVVALSVPGFPVAASIVVGVLFALSIVALTASFLVYIANAKLVAIRIKFLSSGLQDHFSESPILGTLRKGRGASIPLISGQADDSLPNRIGIKKSAEMRVLQKGIGTDYKKYKQHLDRVNND

search 116 ends 

searching for MRAALNTLEFLSSPPSSDPYDDLLQLNKEGFLAGPEEEKQAFFLRVERTLAEAPVHPTPFPIEFQKLFDVNPSFLEVVYSNESLDAWEAGCTWITDNRVSIQLRKGFQKASFWFGFFSKEEVLSHEAVHAVRMKFYEPIFEEVLAYSTSKHFWRRFFGPLFRSAGETHFFLFFVLFGAFLFPWFPWIGLSCILAPNMFFFFRLFRTQILFRKAKKKIRKLLGIEPLWVLLRLTDREIRLFATQPLAVIEDFARKEKLKSVRWRQIYQSYFT
search 117 ends 

searching for MASEYEHFGNLSPEDHVKEVQDLHKVCKGEPHQTKKGYWYHLTCDAIDCGVVLFFIRTIFFLVPAIPVTSYGKILFATGISWIFYTSCKRAQAAWAYIELTHRNMLQEKKEIETNPEQERIELAVLYANQGFQEPLISQMLDFVCSDSSLLLSTMLREELHIQLEDYPHPLKQGNVKALGGILGLLLFAPITLAVSYTIAAILASFMIGVLFAVKTRLIKNAITPAIVWGVGMFITAISLCCSLIRLF
search 118 ends 

searching for MNSGMFPFTFFLLYICLGMLTAYLANKKNRNLIGWFLAGMFFGIFAIIFLLILPPLPSSTQDNRSMDQQDSEEFLLQNTLEDSEIISIPDTMNQIAIDTEKWFYLNKDCTNVGPISIVQLTAFLKECKHSPEKGIDPQELWVWKKGMPNWEKVKNIPELSGTVKDE
search 119 ends 

searching for MTTLPNTCTSNSNSINTFTKDIEMAKQIQGSRKDPLAKTSWIAGLICVVAGVLGLLAIGIGGCSMASGLGLIGAIIAAVVVAVGLCCLVSALCLQVEKSQWWQKEFKSWIEQKSQFRIVMADMLEANQKLQSEVEFLSKGWSDAAAVHKEDVTKYEQVVEKYGEKIMKLYKQTGVLTIEKVNLQ

search 142 ends 

searching for MKFHFPCSEGLIRGFSREFLCCILTDKKIQYPLKNFICVEFIVNSFFGILPRGIPNVGRLSEVAGENKQSLEEREQDKALKLEKKLLAIRKRIKCFCPQQSEISVQAAPLKHTGTFPCSEEELRDISDLFSSLKFFRQQLAQLFFYTPPLNLGWEDFLKFFFAFEKRELGGILFSAGPFESFDRYLYQVNKARPVPVLIATTVSYALQAYCSYINRAPFQEKENFFQLGEAVGIFLKERMVSIALMYKEILDLDNKQYFELCRGIQKSQIVQGEVFHSSTQERDGLNPISVNYDLMGTIAALSVNIDRSCLRFSGSHIFHDDEMAIETLHKGGDVFTFSSLAEFQFSEKRLLHLVSTGRVCPEIIRKKLIKVLLLKKKLSVSLFAGISKLHQRQLN
search 143 ends 

searching for MGIKPHDHGCWGSRGNIFTLQDLDTQQANQSAAASSSSVLKSECAAKVAHCALGFLFGLGFILSIVTFIAAAATLPLGTVTILIMVTQAAFAAALAFKLYDLFKHDVPTCSITSKA
search 144 ends 

searching for MVYFRAHQPRHTPKTFPLEVHHSFSDKHPQIAKAMWITGIALAALSLLAVVACVIAVSAGGAAIPLAVIGGIAAMSGLLSAATIICSAKKALAQRKQKQLKESLPLDNATEHVNYLTSDTSYFNQWASLDALNKQLSQIDLTIQAPEKKLLKEVLGSRYDSINHSIEEISDRFTKMLSLLRLREHFYRGEERYAPYLSPPLLNKNRLLTQITSNMIRMLPKSGGVFSLKANTLSHASRTLYTVLKVALSLGVLAGVAALIIFLPPSLPFIAVIGVSSLALGMASFLMIRGIKYLLEHSPLNRKQLAKDIQKTIIPDVLASMVHYQHQLLSHLHETLLDEAITARWSEPFFIEHANLKAKIEDLTKQYDILNAAFNKSLQQDEALRSQLEK

### 5. Interpret the BLAST search result

Open the control result for blast search and check if our true target, **Mycobacterium tuberculosis**, is in the result.

In [5]:
control_file = open(file, "r")
for line in control_file:
    if "Mycobacterium tuberculosis" in line:
        print(line)
print ("No result for Mycobacterium tuberculosis")

No result for Mycobacterium tuberculosis


### 6. Result and Conclusion